In [1]:
!pip freeze | grep pandas

pandas==2.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('web-service/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [33]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(year, month):
    filename= f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    return df

In [32]:
year = 2023
month = 3
f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
f'{year:04d}/{month:02d}_'

'2023/03_'

In [34]:
df = read_data(2023,3)

In [35]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [36]:
y_pred.std()

6.247488852238703

In [37]:
with open('model.bin', 'wb') as f_out: # 'wb' means write-binary
    pickle.dump((dv, model), f_out)

In [38]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [11]:
df_results= df[["ride_id"]]
df_results["predictions"] = y_pred.tolist()

/tmp/ipykernel_33989/1435006717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results["predictions"] = y_pred.tolist()


In [12]:
df_results.size

6632432

In [13]:
df_results.to_parquet(
    "/workspaces/MLOpps24/04-deployment/data/results_dataframe.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3316216 entries, 0 to 3403765
Data columns (total 21 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee            floa